Result from data version 491

## before you run, make sure you have enough RAM

#### Change the below value for whether or not you want to export data to ./output

In [1]:
output_result = False

below are some code rewritten from initial analysis

In [2]:
import re
import json
import pandas as pds
from os import listdir

In [3]:
data_files_location = {}
categories = {}

for f in listdir('./data'):
    if f.endswith('.json'):
        temp = json.load(open(f'./data/{f}'))
        for i in temp['items']:
            categories[i['id']] = i['snippet']['title']
    if f.endswith('.csv'):
        data_files_location[f[:2]] = f'./data/{f}'

if not categories:
    raise FileNotFoundError('can not locate any json category files')
    
if not data_files_location:
    raise FileNotFoundError('not CSV data files are located')

In [4]:
all_dfs = {}

for k, v in data_files_location.items():
    all_dfs[k] = pds.read_csv(v)

data processing code

In [5]:
def phrasing(s: str, k = 1):
    arr = re.findall(r"\w+'*\w+", s.lower())
    
    if len(arr) > k < 1:
        raise ValueError('Invalid array input or k value')
    ret = []
    for i in range(len(arr)+1-k):
        extract = arr[i:i+k]
        ret.append(' '.join(extract))
    return ret

def analyze_dataframe(label: str, df, max_phrase: int = 3):
    ret = {
        'trend category': {},
        'trend title phrase': {},
        'trend channels': {}
    }
    
    temp = {}
    channels_data = {}
        
    for i in df.itertuples():
        c = categories[str(i.categoryId)]
        
        try:
            temp[i.channelId]
        except KeyError:
            temp[i.channelId] = i.channelTitle
            
        try:
            channels_data[i.channelId] += 1
        except KeyError:
            channels_data[i.channelId] = 1
        
        try:
            ret['trend category'][c] += 1
        except KeyError:
            ret['trend category'][c] = 1
            
        result = phrasing(i.title, max_phrase)

        for r in result:
            try:
                ret['trend title phrase'][c]
            except KeyError:
                ret['trend title phrase'][c] = {}

            try:
                ret['trend title phrase'][c][r] += 1
            except KeyError:
                ret['trend title phrase'][c][r] = 1
                
    for k, v in channels_data.items():
        t = temp[k]
        try:
            ret['trend channels'][t]
        except KeyError:
            ret['trend channels'][t] = v
        else:
            ret['trend channels'][f'{t} ({k})'] = v
    
    # code from: https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
    ret['trend category'] = dict(sorted(ret['trend category'].items(), key=lambda item: item[1], reverse=True))
    ret['trend channels'] = dict(sorted(ret['trend channels'].items(), key=lambda item: item[1], reverse=True))
    
    t = 'trend title phrase'
    for k in ret[t].keys():
        ret[t][k] = dict(sorted(ret[t][k].items(), key=lambda item: item[1], reverse=True))
        
    ret['rows'] = len(df)
    ret['columns'] = len(df.columns)
    
    return ret

In [6]:
output = {}

for k, v in all_dfs.items():
    output[k] = analyze_dataframe(k, v)


export result to ./output in *_analysis.json format

In [7]:
if output_result:
    for k, v in output.items():
        with open(f'./output/{k}_analysis.json', 'w') as f:
            json.dump(v, f, indent=4)

code for outputting result for display

In [8]:
def print_dict_sort(d, limit=-1, indent: str = ''):
    count = 0
    # code from: https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
    for i, k in enumerate(d):
        print(f'{indent}{d[k]}:\t{k}')
        if limit != -1:
            if i >= limit:
                break

def print_output(out, limit=50):
    print('Numbers of Trend Video by Category:')
    print_dict_sort(out['trend category'], indent='\t')
    print('Channels and trend video count:')
    print_dict_sort(out['trend channels'], limit, indent='\t')
    
    print('\nTrend Video Title Separated By Category:')
    for k, v in out['trend title phrase'].items():
        print(f'\t{k}:')
        print_dict_sort(v, limit, '\t\t')
        if limit != -1:
            print('\t\t...')

### Analysis Data For Brazil

In [9]:
print_output(output['BR'])

Numbers of Trend Video by Category:
	25621:	Entertainment
	25346:	Music
	13468:	Sports
	13180:	People & Blogs
	7360:	Gaming
	5126:	Comedy
	1587:	Education
	1536:	News & Politics
	1493:	Science & Technology
	1022:	Film & Animation
	793:	Howto & Style
	772:	Autos & Vehicles
	314:	Travel & Events
	144:	Pets & Animals
	31:	Nonprofits & Activism
Channels and trend video count:
	1137:	ge
	1009:	Felipe Neto
	617:	FutParódias
	578:	MasterChef Brasil
	539:	De Placa
	505:	WebTVBrasileira
	494:	Esporte Interativo
	490:	Free Fire - Brasil
	470:	Invento na Hora
	445:	A Fazenda
	439:	NOW UNITED
	438:	LOUD
	427:	AM3NlC
	421:	FOX Sports Brasil
	407:	NBA Brasil
	402:	Dinah Moraes
	385:	UOL Esporte
	378:	Me poupe!
	374:	Carol Borba
	371:	Jogo Aberto
	365:	NFA CHANNEL
	359:	São Paulo FC
	340:	Virgínia Fonseca
	320:	Renato Garcia
	309:	Big Hit Labels
	306:	Porta dos Fundos
	297:	Hora do Faro
	295:	LoL eSports BR
	280:	BANGTANTV
	276:	TV Palmeiras/FAM
	275:	SMTOWN
	270:	Netflix Brasil
	267:	Bruno Garcia
	2

### Analysis Data For Canada

In [10]:
print_output(output['CA'])

Numbers of Trend Video by Category:
	20592:	Entertainment
	16496:	Gaming
	13475:	Music
	11421:	Sports
	8918:	People & Blogs
	6323:	Comedy
	3963:	Science & Technology
	3733:	Howto & Style
	3447:	News & Politics
	3218:	Film & Animation
	2685:	Education
	2116:	Autos & Vehicles
	677:	Travel & Events
	586:	Pets & Animals
	94:	Nonprofits & Activism
Channels and trend video count:
	508:	NBA
	379:	NHL
	342:	MrBeast Gaming
	335:	SSundee
	308:	MrBeast
	283:	DAZN Canada
	278:	Dude Perfect
	238:	The Game Theorists
	224:	Fortnite
	224:	SMTOWN
	223:	Big Hit Labels
	222:	FORMULA 1
	222:	ESPN
	219:	Veritasium
	216:	Tom Scott
	212:	BLACKPINK
	209:	JYP Entertainment
	208:	The Try Guys
	205:	SpaceX
	203:	First We Feast
	202:	NFL
	198:	beIN SPORTS USA
	192:	Nintendo
	190:	Apex Legends
	190:	Marvel Entertainment
	189:	SPORTSNET
	187:	UFC - Ultimate Fighting Championship
	183:	The Food Theorists
	183:	Warner Bros. Pictures
	175:	LazarBeam
	173:	SSSniperWolf
	172:	Kurzgesagt – In a Nutshell
	171:	The Film Th

		10:	huge bear off
		10:	bear off wall
		10:	off wall to
		10:	wall to protect
		10:	to protect her
		10:	protect her dogs
		10:	coming up from
		10:	up from his
		10:	from his burrow
		8:	avatar floating mountains
		8:	wild horse immediately
		8:	horse immediately recognizes
		8:	immediately recognizes his
		8:	recognizes his girlfriend
		8:	his girlfriend after
		8:	girlfriend after years
		8:	after years apart
		8:	years apart the
		8:	apart the dodo
		8:	the dodo faith
		8:	dodo faith restored
		...
	Nonprofits & Activism:
		14:	global citizen live
		8:	color the spectrum
		8:	the spectrum live
		8:	spectrum live mark
		8:	live mark rober
		8:	mark rober and
		8:	rober and jimmy
		8:	and jimmy kimmel
		7:	global citizen vax
		7:	citizen vax live
		7:	vax live extended
		7:	live extended concert
		7:	extended concert only
		7:	concert only on
		7:	only on youtube
		6:	was hacked but
		6:	hacked but now
		6:	but now i'm
		6:	now i'm back
		5:	returning to youtube
		5:	to youtube our

### Analysis Data For Germany

In [11]:
print_output(output['DE'])

Numbers of Trend Video by Category:
	25614:	Entertainment
	17753:	Music
	11549:	Sports
	10981:	People & Blogs
	9278:	Gaming
	4120:	Autos & Vehicles
	4003:	Howto & Style
	3584:	Science & Technology
	3383:	News & Politics
	2832:	Comedy
	2005:	Education
	1525:	Film & Animation
	588:	Travel & Events
	441:	Pets & Animals
	87:	Nonprofits & Activism
Channels and trend video count:
	867:	JP Performance
	864:	Berlin - Tag & Nacht
	645:	Köln 50667
	624:	Sky Sport HD
	541:	tagesschau
	520:	Krass Schule - Die jungen Lehrer
	450:	Goal Deutschland
	426:	DAZN Bundesliga
	389:	Varion
	329:	DAZN UEFA Champions League
	314:	LUCA
	272:	Sportschau
	269:	DAZN Länderspiele
	256:	BibisBeautyPalace
	231:	Freshtorge
	222:	FORMULA 1
	210:	NFL
	208:	DFB
	207:	Fritz Meinecke
	207:	Joko & Klaas
	205:	Team Harrison
	204:	MrBeast
	193:	Jindaouis
	192:	DER SPIEGEL
	187:	DAZN FA Cup & Carabao Cup
	184:	DAZN UEFA Europa League
	184:	ZDFsport
	181:	GRIP - Das Motormagazin
	179:	Borussia Dortmund
	179:	SMTOWN
	178:	Dude 

		17:	rennen 2021 nürburgring
		16:	porsche gt2 rs
		16:	audi rs tron
		16:	rs tron gt
		16:	im pace museum
		16:	teil philipp kaess
		16:	steel buddies throwbackthursday
		16:	buddies throwbackthursday dmax
		...
	Science & Technology:
		96:	smarter every day
		66:	iphone 13 pro
		65:	iphone 12 pro
		46:	was ist neu
		42:	apple iphone 12
		39:	update ist da
		37:	ist da was
		37:	da was ist
		36:	ios 14 update
		36:	14 update ist
		34:	ganze folge terra
		34:	high altitude flight
		31:	apple watch series
		29:	samsung galaxy s21
		28:	m1 max macbook
		27:	12 pro max
		26:	ist neu über
		25:	altitude flight test
		25:	13 pro max
		24:	das müsst ihr
		24:	müsst ihr wissen
		24:	galaxy s21 ultra
		24:	iphone 13 13
		22:	meine meinung ersteindruck
		22:	meinung ersteindruck deutsch
		22:	ersteindruck deutsch swagtab
		21:	s21 s21 ultra
		21:	max macbook pro
		20:	galaxy s21 s21
		19:	iphone 12 12
		19:	13 13 pro
		18:	leben auf der
		18:	auf der venus
		18:	wish com gaming
		18:	com gamin

### Analysis Data For France

In [12]:
print_output(output['FR'])

Numbers of Trend Video by Category:
	23823:	Entertainment
	20755:	Music
	10292:	Sports
	9716:	Comedy
	9674:	Gaming
	4997:	People & Blogs
	3934:	Howto & Style
	3404:	Science & Technology
	2765:	News & Politics
	2700:	Autos & Vehicles
	2639:	Education
	2164:	Film & Animation
	551:	Travel & Events
	292:	Pets & Animals
	85:	Nonprofits & Activism
Channels and trend video count:
	1040:	beIN SPORTS France
	603:	HugoDécrypte
	573:	Michou
	556:	Ligue 1 Uber Eats
	545:	SQUEEZIE
	497:	Amixem
	494:	RMC Sport
	466:	Fuze III
	404:	JOYCA
	396:	CANAL+ Sport
	355:	SQUEEZIE GAMING
	343:	Romain Dot Live
	331:	Léna Situations
	322:	MichouOff
	322:	Fédération Française de Football
	315:	Benjamin Verrecchia
	311:	Pierre Croce
	309:	LeBouseuh
	302:	Mcfly et Carlito la chaîne secondaire
	296:	Inoxtag
	291:	Mastu
	284:	Valouzz - Inoxtag 3.0
	283:	Mcfly et Carlito
	273:	France tv sport
	268:	Pidi
	263:	Poisson Fécond
	251:	Tibo InShape
	245:	Lucas Studio
	236:	Vilebrequin
	235:	Booska-P.com
	234:	Camille LV
	22

	Travel & Events:
		25:	l'homme qui survécu
		23:	vanlife voyage voyages
		14:	van fourgon aménagé
		14:	fourgon aménagé camping
		14:	aménagé camping car
		11:	une grosse surprise
		11:	grosse surprise en
		11:	surprise en afrique
		11:	en afrique requin
		11:	afrique requin bouledogue
		11:	requin bouledogue cyril
		11:	bouledogue cyril chauquet
		10:	the vadrouilleurs for
		10:	vadrouilleurs for rêveur
		10:	du grand nord
		9:	combien me coûte
		9:	je teste le
		9:	qui survécu jours
		8:	encerclé par des
		8:	premiers problèmes mecaniques
		8:	problèmes mecaniques révision
		8:	mecaniques révision avant
		8:	révision avant le
		8:	avant le contrôle
		8:	le contrôle technique
		8:	contrôle technique du
		8:	technique du fourgon
		8:	du fourgon aménagé
		8:	fourgon aménagé vanlife
		7:	teste le jeu
		7:	le jeu de
		7:	jeu de société
		7:	de société minecraft
		7:	société minecraft officiel
		7:	sous la glace
		7:	en amazonie cyril
		7:	amazonie cyril chauquet
		7:	ces requins attaquen

### Analysis Data For Great Britain

In [13]:
print_output(output['GB'])

Numbers of Trend Video by Category:
	22203:	Entertainment
	18006:	Sports
	14344:	Gaming
	12884:	Music
	9003:	People & Blogs
	4965:	Comedy
	3479:	Science & Technology
	2767:	News & Politics
	2588:	Howto & Style
	2511:	Film & Animation
	2080:	Education
	1885:	Autos & Vehicles
	529:	Travel & Events
	472:	Pets & Animals
	79:	Nonprofits & Activism
Channels and trend video count:
	1173:	The United Stand
	1104:	Sky Sports Football
	802:	BT Sport
	501:	WWE
	416:	FORMULA 1
	394:	Jay's Virtual Pub Quiz
	341:	AFTV
	332:	MoreSidemen
	297:	MrBeast
	296:	SidemenReacts
	290:	ITV Sport
	273:	Sidemen
	273:	MrBeast Gaming
	262:	Sky Sports Boxing
	261:	Dude Perfect
	261:	England
	239:	carwow
	238:	BT Sport Boxing
	236:	Mark Goldbridge That's Football
	228:	SSundee
	224:	GRM Daily
	221:	Fortnite
	220:	LaLiga Santander
	216:	Tom Scott
	210:	SPFL
	209:	BBC Sport
	208:	Anastasia Kingsnorth
	205:	Veritasium
	204:	Miniminter
	203:	SpaceX
	200:	UFC - Ultimate Fighting Championship
	195:	TommyInnit
	192:	Serie A

		23:	insane engineering of
		23:	engineering of the
		22:	apple watch series
		22:	iphone 12 vs
		22:	power loader part
		22:	in 11 minutes
		21:	is out what's
		21:	out what's new
		20:	ingenuity mars helicopter
		19:	the tech chap
		19:	ios 14 beta
		19:	how to make
		19:	12 12 pro
		19:	28 546 fps
		19:	546 fps slow
		19:	tour with elon
		19:	with elon musk
		19:	elon musk part
		18:	samsung galaxy fold
		18:	the truth about
		18:	need to know
		18:	m1 macbook pro
		18:	fps slow motion
		18:	iphone 13 event
		17:	nuclear submarine smarter
		17:	nasa's spacex crew
		17:	spacex crew mission
		17:	2021 official replay
		17:	official replay samsung
		17:	nasa perseverance rover
		17:	event in 11
		17:	m1 max macbook
		17:	max macbook pro
		...
	Travel & Events:
		108:	us vs uk
		22:	new northern line
		17:	the new northern
		13:	tube trains on
		13:	trains on the
		12:	on the isle
		12:	the isle of
		12:	isle of wight
		12:	northern line extension
		10:	this is the
		10:	is the end
		1

### Analysis Data For India

In [14]:
print_output(output['IN'])

Numbers of Trend Video by Category:
	37058:	Entertainment
	14826:	Music
	11689:	People & Blogs
	7124:	Comedy
	3544:	Gaming
	3377:	Science & Technology
	3160:	News & Politics
	3160:	Sports
	3138:	Howto & Style
	2079:	Education
	1861:	Film & Animation
	567:	Autos & Vehicles
	532:	Travel & Events
	41:	Nonprofits & Activism
	26:	Pets & Animals
Channels and trend video count:
	1317:	Vijay Television
	1110:	Colors TV
	986:	ZEE5
	928:	SAB TV
	824:	Asianet
	729:	Crazy XYZ
	591:	Taarak Mehta Ka Ooltah Chashmah
	561:	T-Series
	540:	Colors Rishtey
	470:	MR. INDIAN HACKER
	406:	Flowers Comedy
	386:	Sun TV
	381:	Star Maa
	355:	Sourav Joshi Vlogs
	338:	SET India
	324:	Zee TV
	316:	ETV Jabardasth
	311:	Hungry Birds
	269:	Sony PAL
	263:	Rachit Rojha
	253:	Speed Records
	252:	BakLol Video
	250:	England & Wales Cricket Board
	247:	Amazon Prime Video India
	246:	Flying Beast
	243:	mallemalatv
	243:	Dushyant kukreja
	239:	Triggered Insaan
	238:	The MriDul
	236:	A2 Motivation {Arvind Arora}
	234:	Netflix I

		...
	Education:
		154:	challenge hungry birds
		120:	kumar gaurav sir
		101:	daily current affairs
		84:	dr vivek bindra
		75:	for 24 hours
		65:	india hungry birds
		63:	challenge food challenge
		63:	food challenge india
		63:	challenge india hungry
		61:	exam kumar gaurav
		58:	24 hours challenge
		49:	for all exam
		49:	all exam kumar
		45:	madan gowri mg
		44:	for all exams
		43:	by vivek sir
		40:	hours challenge hungry
		40:	all exams kumar
		40:	exams kumar gaurav
		40:	world geography for
		40:	geography for all
		40:	october daily current
		38:	food challenge hungry
		38:	ekadashi vrat katha
		35:	ca rachana ranade
		35:	एक दश कथ
		34:	tamil madan gowri
		32:	nirmohi sir sanskritganga
		29:	इस पर बन
		27:	september daily current
		27:	up tet_super tet
		26:	questions for all
		25:	food challenge food
		25:	dr vikas divyakirti
		22:	2nd 3rd shift
		21:	rich vs normal
		21:	november daily current
		20:	diwali rangoli designs
		20:	for diwali इस
		20:	diwali इस पर
		19:	living

### Analysis Data For Japan

In [15]:
print_output(output['JP'])

Numbers of Trend Video by Category:
	32678:	Entertainment
	14356:	People & Blogs
	11961:	Music
	11078:	Gaming
	7363:	Sports
	5039:	Film & Animation
	4745:	Comedy
	4288:	Howto & Style
	1809:	Autos & Vehicles
	1567:	News & Politics
	1536:	Pets & Animals
	626:	Education
	493:	Science & Technology
	186:	Travel & Events
	62:	Nonprofits & Activism
Channels and trend video count:
	1359:	東海オンエア
	580:	Fischer's-フィッシャーズ-
	540:	コムドット
	497:	両学長 リベラルアーツ大学
	490:	もちまる日記
	402:	THE FIRST TAKE
	400:	SixTONES
	398:	はじめしゃちょー（hajime）
	386:	4月25日19時オープンのチャンネル
	378:	すとぷりちゃんねる
	373:	48-フォーエイト
	364:	朝倉未来 Mikuru Asakura
	356:	Snow Man
	341:	ジャニーズJr.チャンネル
	327:	ヒカル（Hikaru）
	324:	P丸様。
	309:	RIZIN FIGHTING FEDERATION
	303:	東海オンエアの控え室
	303:	HikakinTV
	301:	NiziU Official
	278:	モンスト（モンスターストライク）公式
	275:	まあたその「ほぼメインチャンネル」
	272:	きまぐれクックKimagure Cook
	271:	仲里依紗です。
	263:	/谷やん谷崎鷹人
	262:	はじめまして松尾です
	260:	JO1
	259:	あめんぼぷらす
	258:	BANGTANTV
	258:	KAI Channel / 朝倉海
	258:	カジサック KAJISAC
	257:	パパラピーズ
	255:	かす
	253:	QuizKnock
	253

		12:	24年ぶりの天体ショー 次は12年後 2021年5月26日
		12:	スーパームーン 皆既月食 total
		12:	lunar eclipse 2021
		12:	eclipse 2021 japan
		12:	2021 japan 2021年5月26日
		12:	japan 2021年5月26日 live配信
		11:	なぜ 愛子さまティアラ作らず借用 ドレス姿初披露へ
		11:	愛子さまティアラ作らず借用 ドレス姿初披露へ 皇室ちょっといい話
		11:	ドレス姿初披露へ 皇室ちょっといい話 44
		11:	皇室ちょっといい話 44 2021年11月16日
		10:	天才数学者 人生に必要 算数で十分
		10:	人生に必要 算数で十分 文系でもマスト
		10:	算数で十分 文系でもマスト 理論的に考える癖をつけるため
		10:	文系でもマスト 理論的に考える癖をつけるため 天才数学者
		10:	理論的に考える癖をつけるため 天才数学者 ひろゆきが議論
		10:	天才数学者 ひろゆきが議論 メトロノーム
		10:	ひろゆきが議論 メトロノーム アベプラ
		10:	メトロノーム アベプラ アベマで放送中
		10:	独自 接種から数時間後に女性が急死 真相究明求める遺族
		10:	接種から数時間後に女性が急死 真相究明求める遺族 消去法でそれしかない
		10:	真相究明求める遺族 消去法でそれしかない 2021年6月10日
		10:	車にスプレーで落書き 度重なる被害に防犯カメラを設置した被害者 カメラに映っていた２人組とは
		10:	生放送中のスタジオに大きな揺れ 地震発生の瞬間 2021年10月7日
		10:	ひろゆきの妻 全て暴露 家に警察来ました
		10:	全て暴露 家に警察来ました web初顔出し
		10:	家に警察来ました web初顔出し 西村ゆか激白
		9:	台湾 公式訪問 中国にno
		9:	公式訪問 中国にno チェコ
		9:	中国にno チェコ 本気
		9:	チェコ 本気 のワケは
		9:	本気 のワケは 後編
		9:	のワケは 後編 2020年9月4日
		9:	報ステ特集 卒業生へ贈る 藤井風さん
		9:	卒業生へ贈る 藤井風さん 旅路
		9:	本来は日本に生息せず 

### Analysis Data For South Korea

In [16]:
print_output(output['KR'])

Numbers of Trend Video by Category:
	33051:	Entertainment
	14243:	People & Blogs
	10002:	Music
	7265:	Sports
	5666:	Comedy
	5378:	News & Politics
	3980:	Howto & Style
	3308:	Film & Animation
	3198:	Gaming
	2177:	Education
	2120:	Pets & Animals
	1892:	Science & Technology
	1315:	Autos & Vehicles
	1015:	Travel & Events
	144:	Nonprofits & Activism
Channels and trend video count:
	622:	짤툰
	607:	SPOTV
	468:	Mnet Official
	455:	총몇명
	451:	파뿌리
	444:	BANGTANTV
	420:	엠뚜루마뚜루 : MBC 공식 채널
	419:	장삐쭈
	411:	워크맨-Workman
	396:	런닝맨 - 스브스 공식 채널
	390:	Big Hit Labels
	389:	스포츠타임
	366:	채널 십오야
	359:	유 퀴즈 온 더 튜브
	351:	SMTOWN
	346:	문명특급 - MMTG
	321:	SBS NOW / SBS 공식 채널
	292:	올리버쌤
	283:	달라스튜디오
	282:	디글 :Diggle
	282:	육식맨
	280:	썰레임 - 미쳐버린 영상툰채널
	278:	웃소 Wootso
	277:	JTBC Entertainment
	276:	시즌비시즌 Season B Season
	275:	JYP Entertainment
	275:	영국남자 Korean Englishman
	275:	MBCNEWS
	273:	JTBC Voyage
	273:	짧은대본 ShortPaper
	272:	이과장
	271:	아는형님 Knowingbros
	269:	딩고 뮤직 / dingo music
	261:	SBS ENTER PLAY
	257:	SBS Catch
	2

		29:	우리 엄마가 제일
		27:	새로운 방법 잊을
		27:	방법 잊을 없는
		24:	다이어트 최고의 운동
		24:	맛있게 담그는 방법
		23:	방법이 최고입니다 새로운
		23:	최고입니다 새로운 방법
		23:	이렇게 만들었더니 1년이
		23:	만들었더니 1년이 행복합니다
		23:	1년이 행복합니다 빨리
		23:	행복합니다 빨리 한단
		23:	빨리 한단 사오세요
		23:	한단 사오세요 평생
		23:	사오세요 평생 써먹는
		23:	평생 써먹는 레시피
		22:	이렇게 맛있는지 알았더라면
		22:	맛있는지 알았더라면 생으로
		22:	알았더라면 생으로 먹지
		22:	생으로 먹지 않았을
		22:	먹지 않았을 겁니다
		22:	않았을 겁니다 새로운
		22:	많이 배우네요 욕심이
		22:	배우네요 욕심이 나는
		22:	욕심이 나는 요리입니다
		21:	뱃살 아랫뱃살 최고의
		21:	아랫뱃살 최고의 운동
		21:	방법이 최고입니다 쉽고
		21:	최고입니다 쉽고 간단
		21:	쉽고 간단 새로운
		21:	간단 새로운 방법
		21:	음식으로 변합니다 정말
		20:	그냥 버리지 마세요
		20:	계속 저어줄 필요없어요
		20:	저어줄 필요없어요 냄비에
		20:	필요없어요 냄비에 넣고
		20:	냄비에 넣고 10분이면
		20:	넣고 10분이면 매일
		20:	10분이면 매일 먹었더니
		...
	Autos & Vehicles:
		65:	블랙박스 모음 zip
		65:	모음 zip 몇대몇
		65:	zip 몇대몇 블랙박스
		43:	몇대몇 블랙박스 eng
		33:	블랙박스 eng sub
		24:	운전자 대부분이 모르는
		24:	hidden features of
		24:	features of my
		24:	of my cars
		24:	my cars 차업차득
		19:	all new rexton
		18:	람보르기니랑 덤프트럭 사고
		18:	두번째 블랙박스 모음
		17:	몇대몇 블랙박스 모음집
		17:	블랙박스

### Analysis Data For Mexico

In [17]:
print_output(output['MX'])

Numbers of Trend Video by Category:
	23917:	Music
	22816:	Entertainment
	12529:	People & Blogs
	12011:	Gaming
	9117:	Sports
	4392:	Comedy
	3768:	News & Politics
	3272:	Howto & Style
	2066:	Film & Animation
	1160:	Autos & Vehicles
	836:	Science & Technology
	835:	Travel & Events
	748:	Education
	120:	Nonprofits & Activism
	12:	Pets & Animals
Channels and trend video count:
	897:	TUDN México
	593:	Luisito Comunica
	582:	Marca Claro
	555:	Cracks
	468:	Free Fire - LATAM
	462:	Más SKabeche
	462:	Kimberly Loaiza
	453:	FOX Sports MX
	449:	Andrés Manuel López Obrador
	426:	BANGTANTV
	386:	Antrax
	385:	ESPN Deportes
	357:	LosPolinesios
	349:	JYP Entertainment
	349:	Big Hit Labels
	344:	La Capital
	323:	LaLiga Santander
	311:	TheGrefg
	292:	SMTOWN
	283:	Distrito Comedia
	281:	JuegaGerman
	276:	BLACKPINK
	273:	TheDonato
	271:	Las Estrellas
	269:	Mis Pastelitos
	266:	MrBeast en español
	245:	Kenia Os
	243:	Sandra Cires Art
	241:	TV Azteca Deportes
	239:	YOLO AVENTURAS
	239:	BERTH OH!
	231:	JUCA
	2

		34:	línea 12 del
		34:	12 del metro
		34:	cámara de diputados
		34:	la casa de
		34:	casa de los
		34:	de los famosos
		33:	de prensa covid19
		32:	contra covid 19
		32:	noticias con francisco
		32:	con francisco zea
		32:	de enero de
		32:	enero de 2021
		31:	de covid 19
		30:	de estados unidos
		30:	de ee uu
		28:	las elecciones en
		28:	canal de suez
		27:	la casa blanca
		27:	en vivo se
		26:	noticias con juan
		26:	con juan carlos
		26:	juan carlos valerio
		25:	terremoto de magnitud
		25:	sismo de magnitud
		25:	del covid 19
		25:	cambio de titular
		25:	de titular en
		24:	presidente electo de
		24:	millones de pesos
		24:	socavón de puebla
		23:	de la pandemia
		23:	en reino unido
		23:	el centro de
		22:	al rojo vivo
		22:	rojo vivo telemundo
		22:	de la cdmx
		22:	estrella de belén
		22:	vivo se cae
		22:	se cae la
		22:	cae la barda
		...
	Comedy:
		162:	daniel el travieso
		102:	la india yuridia
		79:	el super trucha
		49:	el súper trucha
		48:	vale la pena
		42:	con el c

### Analysis Data For Russia

In [18]:
print_output(output['RU'])

Numbers of Trend Video by Category:
	16479:	People & Blogs
	15829:	Entertainment
	12083:	News & Politics
	8093:	Music
	7814:	Sports
	7248:	Film & Animation
	6564:	Comedy
	6512:	Gaming
	5934:	Autos & Vehicles
	2850:	Howto & Style
	2801:	Science & Technology
	2201:	Nonprofits & Activism
	899:	Education
	731:	Travel & Events
	603:	Pets & Animals
Channels and trend video count:
	919:	Матч ТВ
	906:	Телеканал Дождь
	890:	Россия 24
	793:	Соловьёв LIVE
	774:	Первый канал
	729:	Анатолий Шарий
	719:	A4
	494:	Россия 1
	473:	НТВ
	427:	Вести FM
	420:	Lesha Maisak
	401:	Эхо Москвы
	344:	Навальный LIVE
	306:	Ozodivideo
	289:	Трум Трум
	283:	Comedy Club
	276:	Редакция
	276:	МУЗЫКА
	265:	PRO Английский футбол
	248:	Телеканал ПЯТНИЦА
	237:	Magic Five
	232:	Дюшес
	230:	Телеканал ТНТ
	229:	Настоящее Время
	227:	Виндяй
	219:	Telesport
	217:	Вечерний Ургант
	207:	UFC Russia
	202:	Lady Diana
	200:	Ксения Собчак
	193:	varlamov
	193:	Детектив Войд
	192:	Александр Невзоров
	191:	marazm
	187:	OneTwo
	186:	Hardco

		387:	russische musik 2021
		371:	2021 russische musik
		358:	лучшие песни 2021
		350:	2021 лучшие песни
		328:	2021 топ музыка
		323:	премьера клипа 2021
		260:	новинки музыки 2021
		246:	2021 новинки музыки
		245:	песни 2021 russische
		241:	хиты 2021 топ
		217:	музыки 2021 лучшие
		183:	премьера клипа 2020
		141:	зажигательные песни аж
		141:	песни аж до
		141:	аж до мурашек
		141:	до мурашек остановись
		141:	мурашек остановись постой
		141:	остановись постой сергей
		141:	постой сергей орлов
		133:	russische musik 2020
		127:	2021 русская музыка
		119:	2020 russische musik
		119:	русская музыка 2021
		118:	2020 лучшие песни
		115:	лучшие песни 2020
		106:	хиты 2021 лучшие
		98:	новинки музыки 2020
		97:	2020 новинки музыки
		93:	музыка 2021 russische
		91:	топ музыка июль
		91:	музыка июль 2021
		86:	2020 топ музыка
		85:	хиты 2020 топ
		83:	песни 2021 топ
		68:	песни 2020 russische
		64:	русская музыка 2020
		63:	2020 русская музыка
		57:	official lyric video
		56:	музыки 2020 л

		5:	мне суде тренинг
		5:	суде тренинг спарта
		5:	шоу маска на
		5:	маска на нтв
		5:	если вы попали
		5:	черная плесень страшилка
		5:	плесень страшилка или
		5:	страшилка или реальная
		5:	или реальная угроза
		5:	исчезнет сразу томаты
		5:	сразу томаты вмиг
		5:	томаты вмиг заплодоносят
		5:	вмиг заплодоносят подкормите
		5:	заплодоносят подкормите июне
		5:	18 история artifex
		4:	ответы на вопросы
		4:	məhv edi ldi
		4:	на один день
		4:	самом деле была
		4:	деле была жизнь
		4:	какой была земля
		4:	клим жуков про
		4:	причин почему вам
		4:	почему вам бы
		4:	вам бы не
		4:	бы не понравилось
		4:	не понравилось быть
		4:	понравилось быть пираткой
		...
	Travel & Events:
		32:	как люди живут
		28:	орёл решка девчата
		21:	орёл решка чудеса
		21:	решка чудеса света
		20:	чудеса света eng
		20:	света eng rus
		20:	eng rus sub
		18:	путешествуй по украине
		15:	сезон 2020 серия
		15:	жизнь других eng
		14:	орёл решка 10
		14:	решка 10 лет
		12:	раскопки юрий гагарин
		12:	по украи

### Analysis Data For USA

In [19]:
print_output(output['US'])

Numbers of Trend Video by Category:
	19621:	Entertainment
	17829:	Music
	16927:	Gaming
	10629:	Sports
	8480:	People & Blogs
	5436:	Comedy
	3735:	Film & Animation
	3527:	News & Politics
	3499:	Science & Technology
	3172:	Howto & Style
	2178:	Education
	1751:	Autos & Vehicles
	515:	Pets & Animals
	404:	Travel & Events
	88:	Nonprofits & Activism
Channels and trend video count:
	626:	NFL
	611:	NBA
	516:	SSSniperWolf
	398:	MrBeast Gaming
	384:	SSundee
	337:	MrBeast
	328:	ESPN
	311:	Saturday Night Live
	310:	Champions League on CBS Sports
	299:	Dude Perfect
	284:	NBC Sports
	272:	Fortnite
	262:	The Game Theorists
	236:	beIN SPORTS USA
	235:	SpaceX
	232:	Veritasium
	222:	Nintendo
	218:	The Try Guys
	216:	Apex Legends
	214:	The Film Theorists
	213:	The Food Theorists
	211:	Big Hit Labels
	207:	First We Feast
	205:	SMTOWN
	205:	BLACKPINK
	205:	ESPN FC
	203:	BANGTANTV
	200:	Warner Bros. Pictures
	200:	Marvel Entertainment
	198:	DAZN USA
	196:	LazarBeam
	192:	WWE
	191:	Skip and Shannon: UNDISPUTE

		23:	takeover of afghanistan
		23:	gabby petito case
		22:	2020 election results
		22:	10 news first
		21:	joe biden and
		20:	donald trump and
		20:	trump supporters storm
		20:	prince philip's funeral
		19:	live 2020 election
		19:	harry and meghan
		18:	60 minutes australia
		18:	the 60 minutes
		18:	president elect joe
		18:	elect joe biden
		17:	speech at the
		17:	biden addresses the
		17:	addresses the nation
		17:	fox6 news milwaukee
		17:	in suez canal
		17:	mass shooting in
		17:	are so expensive
		16:	supreme court confirmation
		16:	joe biden wins
		16:	biden and kamala
		16:	princess diana statue
		16:	kabul airport as
		15:	the white house
		15:	2020 presidential debate
		15:	debate between donald
		15:	between donald trump
		15:	trump and joe
		15:	and joe biden
		15:	2020 presidential election
		...
	Sports:
		581:	full game highlights
		353:	highlights nfl 2020
		283:	week highlights nfl
		219:	highlights ucl on
		219:	ucl on cbs
		202:	on cbs sports
		198:	extended h